<a href="https://colab.research.google.com/github/fedjkeee/NLP_-Deeplearning/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [2]:
pip install sentencepiece

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import tokenization
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


– id: уникальный номер сообщения в системе twitter; 

– tdate: дата публикации сообщения (твита); 
– tmane: имя пользователя, опубликовавшего сообщение; 
– ttext:  текст сообщения (твита);
– ttype: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);
– trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
– tfav: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
– tstcount: число всех сообщений пользователя в сети twitter;
– tfol: количество фоловеров пользователя (тех людей, которые читают пользователя);
– tfrien: количество друзей пользователя (те люди, которых читает пользователь);
– listcount: 

In [5]:
columns=['id','tdate','tmane','ttext','ttype','terp','trtw','tfav','tstcount','tfol','tfrien','listcount']

In [6]:
df_p = pd.read_csv("/content/drive/MyDrive/positive.csv",sep=";",header=None)
df_n = pd.read_csv("/content/drive/MyDrive/negative.csv",sep=";",header=None)

In [7]:
df_p.columns=columns
df_n.columns=columns

In [8]:
df_p.head()

,id,tdate,tmane,ttext,ttype,terp,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [9]:
df_n.head()

,id,tdate,tmane,ttext,ttype,terp,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [10]:
df=pd.concat([df_n,df_p])

In [11]:
df['ttype'].value_counts()

 1    114911
-1    111923
Name: ttype, dtype: int64

In [12]:
df.head()

,id,tdate,tmane,ttext,ttype,terp,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [13]:
X = df['ttext']
y = df['ttype']

In [19]:
#Разделим выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y )
print("X_train dataset: ", X_train.shape)
print("y_train dataset: ", y_train.shape)
print("X_test dataset: ", X_test.shape)
print("y_test dataset: ", y_test.shape)

X_train dataset:  (181467,)
y_train dataset:  (181467,)
X_test dataset:  (45367,)
y_test dataset:  (45367,)


In [14]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [15]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    #clf_output = sequence_output[:, 0, :]
    clf_output = sequence_output[:1]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [16]:
%%time
#module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
#bert_layer = hub.KerasLayer(module_url, trainable=True)
bert_layer = hub.KerasLayer(hub.load('/content/drive/MyDrive/rubert_tf20/'), trainable=True)

CPU times: user 4.85 s, sys: 1.62 s, total: 6.48 s
Wall time: 7.51 s


In [29]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer_2 = hub.KerasLayer(module_url, trainable=True)

INFO:absl:Using /tmp/tfhub_modules to cache modules.


In [17]:
#vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
#vocab_file=output
#do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer('/content/drive/MyDrive/rubert_tf20/vocab.txt')#, do_lower_case)

In [20]:
train_input = bert_encode(X_train[:10000].values, tokenizer, max_len=160)
test_input = bert_encode(X_test.values, tokenizer, max_len=160)
train_labels = y_train[:10000].values

In [63]:
bert_layer.

AttributeError: ignored

In [30]:
model = build_model(bert_layer_2, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [97]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=10,
    callbacks=[checkpoint],
    batch_size=16
)

Epoch 1/10
  7/500 [..............................] - ETA: 12:18 - loss: 0.7481 - accuracy: 0.1389

KeyboardInterrupt: ignored

In [64]:
!pip install transformers

In [22]:
rubert_pytorch_path = '/content/drive/MyDrive/rubert_cased_L-12_H-768_A-12_pt/'
from transformers import TFBertForPreTraining

In [23]:
model_2 = TFBertForPreTraining.from_pretrained(rubert_pytorch_path, from_pt=True)

All PyTorch model weights were used when initializing TFBertForPreTraining.

All the weights of TFBertForPreTraining were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.


In [24]:
model_2.layers

In [41]:
model_2.summary()

Model: "tf_bert_for_pre_training"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
_________________________________________________________________
nsp___cls (TFBertNSPHead)    multiple                  1538      
_________________________________________________________________
mlm___cls (TFBertMLMHead)    multiple                  92920059  
Total params: 178,566,653
Trainable params: 178,566,653
Non-trainable params: 0
_________________________________________________________________


In [72]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model_2.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=10,
    callbacks=[checkpoint],
    batch_size=16
)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/10


ValueError: ignored

In [71]:
model_2.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
from transformers import BertTokenizer, TFBertForPreTraining

#tokenizer = BertTokenizer.vocab_files_names(VOCAB_FILES_NAMES)
#input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
#outputs = model(input_ids)
#prediction_scores, seq_relationship_scores = outputs[:2]

In [37]:
VOCAB_FILES_NAMES = {"vocab_file": "/content/drive/MyDrive/rubert_tf20/vocab.txt"}

In [38]:
tokenizer = BertTokenizer(vocab_file="/content/drive/MyDrive/rubert_tf20/vocab.txt")

In [42]:
input_ids = tf.constant(tokenizer.encode(X_train[:100], add_special_tokens=True))[None, :]  # Batch size 1

ValueError: ignored

In [40]:
input_ids.shape

TensorShape([1, 8])

In [82]:
outputs = model_2(input_ids)

In [83]:
prediction_scores, seq_relationship_scores = outputs[:2]

In [94]:
outputs[:1]

(<tf.Tensor: shape=(1, 8, 119547), dtype=float32, numpy=
 array([[[  0.2930078 ,   0.44601852,   0.16122255, ...,   3.0723736 ,
            3.2276611 ,   1.4608006 ],
         [ -4.595234  ,  -4.2776637 ,  -4.2249513 , ...,  -2.0372229 ,
           -1.3344482 ,  -0.19722259],
         [ -7.813592  ,  -7.6926184 ,  -7.336805  , ..., -10.159516  ,
           -4.5923114 ,  -5.316151  ],
         ...,
         [ -6.02838   ,  -7.3205166 ,  -7.3266463 , ...,  -7.3122015 ,
           -1.6044233 ,  -2.3377113 ],
         [ -8.7342005 , -10.145207  ,  -9.420223  , ...,  -8.696537  ,
           -4.2962904 ,  -2.3754766 ],
         [ -6.186378  ,  -7.0497575 ,  -7.4318247 , ...,  -6.767234  ,
           -5.7831545 ,  -2.35348   ]]], dtype=float32)>,)

In [92]:
type(model_2)

transformers.models.bert.modeling_tf_bert.TFBertForPreTraining

In [54]:
encoded_sen = tokenizer(X_train[:10].tolist())
encoded_lab = y_train[:10]

In [53]:
encoded_sen

{'input_ids': [[101, 168, 254, 6181, 90138, 12099, 49014, 8735, 11382, 40743, 1469, 9627, 3247, 57085, 3296, 30046, 158, 120, 102], [101, 168, 13995, 269, 12567, 257, 11318, 59730, 137, 249, 1699, 12929, 6818, 3299, 52095, 869, 9680, 12952, 110909, 132, 16462, 3998, 13930, 6186, 23086, 130, 1777, 132, 132, 132, 851, 4638, 6164, 27995, 3629, 15662, 122, 102], [101, 266, 271, 168, 13567, 11101, 11536, 230, 10992, 20189, 233, 156, 102472, 18295, 4099, 24437, 883, 63792, 898, 79752, 120, 120, 24060, 1714, 855, 47740, 625, 66244, 1699, 33672, 898, 132, 104157, 887, 14018, 120, 120, 120, 246, 12039, 263, 156, 134, 134, 270, 132, 11682, 134, 240, 251, 151, 10935, 255, 245, 153, 21741, 102], [101, 266, 271, 168, 11242, 21741, 22731, 10874, 13519, 156, 168, 12830, 11579, 263, 230, 16650, 139, 239, 168, 28993, 40366, 3808, 230, 138, 239, 168, 280, 24719, 11191, 17538, 56716, 612, 44967, 852, 4357, 128, 17874, 5239, 862, 43752, 896, 128, 26035, 8124, 106, 156, 238, 102], [101, 26998, 41893, 3477,

In [55]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model_2.fit(
    encoded_sen, encoded_lab,
    validation_split=0.2,
    epochs=1,
    callbacks=[checkpoint],
    batch_size=16
)

ValueError: ignored

In [56]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

In [59]:
input_ids = tf.constant(tokenizer.encode(X_train[:100], add_special_tokens=True))[None, :]  # Batch size 1

ModuleAttributeError: ignored